In [1]:
from datetime import datetime, timezone, timedelta
import time
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import sys
import xarray as xr
sys.path.append('../sdpm_py_util')
import ocn_functions as ocnfuns
import plotting_functions as pltfuns
import atm_functions as atmfuns
import init_funs as initfuns
import swan_functions as swanfuns
import river_functions as rivfuns
import grid_functions as grdfuns
from get_PFM_info import get_PFM_info
from util_functions import s_coordinate_4
from util_functions import display_timing_info
import observation_functions as obsfuns

from make_LV4_dotin_and_SLURM import make_LV4_dotin_and_SLURM
from make_LV4_coawst_dotins_dotsb import make_LV4_coawst_dotins_dotsb
from run_slurm_LV4 import run_slurm_LV4
import glob


import pickle
import os
import pprint
import subprocess



/home/mspydell/models/PFM_root/PFM/driver/../sdpm_py_util/ocn_functions.py:30: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater


In [3]:
fn_r = 'LV1_ocean_rst_202411190600_202411211800.nc'
fn_s = 'LV4_swan_rst_202412141200_027.dat-001'

print(fn_s[0:13])
print(fn_s[13:23])
print(fn_s[26:29])
print(fn_s[0:-4])

print(fn_r[14:26])



LV4_swan_rst_
2024121412
027
LV4_swan_rst_202412141200_027.dat
202411190600


In [2]:
PFM_edit = dict()
PFM_edit['lv4_swan_use_rst'] = 1
initfuns.edit_and_save_PFM(PFM_edit)



PFM info was edited and resaved


In [2]:
initfuns.restart_setup('LV4')

PFM is set to do a forecast from...
2024-12-16 06:00:00
going to restart LV4 from
2024-12-16 06:00:00
looking in /scratch/PFM_Simulations/restart_data/LV4_ocean_rst_202412160600_202412181800.nc for the right restart time...
didnt find the right time in /scratch/PFM_Simulations/restart_data/LV4_ocean_rst_202412160600_202412181800.nc
going to look at a previous forecast restart file...
looking in /scratch/PFM_Simulations/restart_data/LV4_ocean_rst_202412150600_202412171800.nc for the right restart time...
found the time index!
going to restart using /scratch/PFM_Simulations/restart_data/LV4_ocean_rst_202412150600_202412171800.nc and python-index 3
going to restart swan, forecast start time is
2024-12-16 06:00:00
0
2024-12-16 06:00:00
2024-12-14 18:00:00
60
2024-12-17 06:00:00
-1 day, 0:00:00
1
2024-12-16 06:00:00
2024-12-14 18:00:00
36
2024-12-16 06:00:00
0:00:00
we are going to restart swan with the line ...
HOTSTART /scratch/PFM_Simulations/restart_data/LV4_swan_rst_202412141800_036.da

In [2]:
def make_and_update_test_file(fname,dt_sec,ntimes,nfiles):
    fnames = []
    for ii in np.arange(nfiles):
        txt = fname[0:-7] + '_cpu' + str(ii) + fname[-7:]
        fnames.append(txt)

    print(fnames)    

    for a in np.arange(ntimes): # this is the time loop
        a_str = str(a)
        for fname in fnames:
            with open(fname, 'w') as fid:
                fid.write(a_str)
                fid.close()
                
        time.sleep(dt_sec)  # Check every dt_sec second


In [ ]:


fname = '/home/mspydell/models/PFM_root/swan_rst_testing/swan_test_rs.txt'
make_and_update_test_file(fname,15)

In [3]:


fname = '/home/mspydell/models/PFM_root/swan_rst_testing/swan_test_rs.txt'
nfiles = '4'
cmd_list = ['python','-W','ignore','swan_functions.py','check_and_move',fname,'1',nfiles]
os.chdir('../sdpm_py_util')
checking = subprocess.Popen(cmd_list)   
make_and_update_test_file(fname,10,7,int(nfiles))
checking.terminate()
os.chdir('../driver')


['/home/mspydell/models/PFM_root/swan_rst_testing/swan_test_cpu0_rs.txt', '/home/mspydell/models/PFM_root/swan_rst_testing/swan_test_cpu1_rs.txt', '/home/mspydell/models/PFM_root/swan_rst_testing/swan_test_cpu2_rs.txt', '/home/mspydell/models/PFM_root/swan_rst_testing/swan_test_cpu3_rs.txt']
000
File '/home/mspydell/models/PFM_root/swan_rst_testing/swan_test_cpu0_rs.txt' modified. Copying to '/home/mspydell/models/PFM_root/swan_rst_testing/swan_test_cpu0_000_rs.txt'...
000
File '/home/mspydell/models/PFM_root/swan_rst_testing/swan_test_cpu1_rs.txt' modified. Copying to '/home/mspydell/models/PFM_root/swan_rst_testing/swan_test_cpu1_000_rs.txt'...
000
File '/home/mspydell/models/PFM_root/swan_rst_testing/swan_test_cpu2_rs.txt' modified. Copying to '/home/mspydell/models/PFM_root/swan_rst_testing/swan_test_cpu2_000_rs.txt'...
000
File '/home/mspydell/models/PFM_root/swan_rst_testing/swan_test_cpu3_rs.txt' modified. Copying to '/home/mspydell/models/PFM_root/swan_rst_testing/swan_test_cpu

In [4]:
checking.terminate()


In [ ]:

fname = '/home/mspydell/models/PFM_root/swan_rst_testing/swan_test_rs.txt'
cmd_list = ['python','-W','ignore','swan_functions.py','check_and_move',fname,'5']
os.chdir('../sdpm_py_util')
checking = subprocess.Popen(cmd_list)   
make_and_update_test_file(fname,10,5)
checking.terminate()
os.chdir('../driver')


In [13]:
fname_full = 'LV4_swan_rst_202412010600.dat'

fn = fname_full + '-' + str(1).zfill(3)

print(fn[0:-8])
print(fn[-8:])
fn2 = fn[0:-8] + '_006' + fn[-8:]
print(fn2)

print(fn2[13:23])


LV4_swan_rst_202412010600
.dat-001
LV4_swan_rst_202412010600_006.dat-001
2024120106


In [2]:
cmd_list = ['python','-u','-W','ignore','driver_run_LV4_coawst_v1.py'] # this is in the .../PFM/driver/ directory
ret1 = subprocess.run(cmd_list)


now starting the LV4 simulation, setting up...
Current local Time =
2024-12-16 15:43:29.302688
in atmfuns.get_atm_data_on_roms_grid(ATM,RMG)
doing level: 4

ATM on roms grid dict saved with pickle.
return code: 0 (0=good)
done with: atmfuns.get_atm_data_on_roms_grid(ATM,RMG)
this took:
0:00:06.563001


plotting atm and atm on roms grid...
return code: 0 (0=good)
...done with: pltfuns.plot_all_fields_in_one
this took:
0:00:12.267043


driver_run_forcast_LV4: saving LV4_ATM.nc file
file_out is:
/scratch/PFM_Simulations/LV4_Forecast/Forc/LV4_ATM_FORCING.nc
return code: 0 (0=good)
driver_run_forecast_LV4:  done with writing LV4_ATM.nc file.
this took:
0:00:01.311385


driver_run_forcast_LV4: saving LV4_OCN_BC pickle file
OCN_LV3_BC dict loaded with pickle
interpolating z with normal loops.
OCN_LV 4 _BC dict saved with pickle to:  /scratch/PFM_Simulations/LV4_Forecast/Forc/BC_LV4_tmp_file.pkl
return code: 0 (0=good)
done with writing LV4_OCN_BC.pkl file.
this took:
0:02:38.020164


driver_r